In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample
from collections import Counter
import string
from sklearn.dummy import DummyClassifier
from numpy.random import random_integers
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn import tree
from sklearn.svm import LinearSVC

In [2]:
def readTxt(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep='\t')
    
    return df

In [3]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 sep=',')
    
    return df

In [4]:
def toCsvDf(fileName, df):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    df.to_csv(fullFileName, header = False, sep='\t', index = False)

In [5]:
def toCsvNp(fileName, npArray):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')

In [23]:
print(string.punctuation)
print(string.punctuation.replace('\'', ''))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&()*+,-./:;<=>?@[\]^_`{|}~


In [31]:
def createDictionary(trainingSet):
    allWords = list()
    cnt = Counter()
    
    if trainingSet=='IMDB':
        trainingDF = readTxt('\IMDB-train.txt')
    elif trainingSet=='yelp':
        trainingDF = readTxt('\yelp-train.txt')
    
# Replacing !"#$%&()*+,-./:;<=>?@[\]^_`{|}~ with ' ' * 31 (31 spaces, needs to be same length)
# and replacing ' with ' ' (apostrophe with space)
    translator = str.maketrans(string.punctuation.replace('\'', ''), 31*' ', '\'')
    
    for i in range(0, len(trainingDF)):
        allWords.extend(trainingDF.iloc[i,0].translate(translator).lower().split(" "))
    
    for word in allWords:
        cnt[word] +=1
    print(cnt.most_common(10)[1:])
    dictionaryWords = list(zip(*cnt.most_common(10001)[1:]))[0]
    
    dictionary = {}
    for index, key in enumerate(dictionaryWords):
        dictionary[key] = index
#     print(dictionary)
    newArray = np.asarray(cnt.most_common(10000)[1:])
    withIndexVocabArray = np.insert(newArray, 1, range(0,10000),1)
    
    if trainingSet=='IMDB':
        toCsvDf('\IMDB-vocab.txt', pd.DataFrame(withIndexVocabArray))
    elif trainingSet=='yelp':
        toCsvDf('\yelp-vocab.txt', pd.DataFrame(withIndexVocabArray))
        
    return dictionary
createDictionary('IMDB')

[('the', 201173), ('and', 98176), ('a', 97789), ('of', 87191), ('to', 81487), ('is', 64545), ('br', 61429), ('in', 56016), ('it', 47572)]


ValueError: could not broadcast input array from shape (10000,1) into shape (9999,1)

In [7]:
def convertWordsToVector(trainingRow, dictionary, BOWType):
    translator = str.maketrans('','', string.punctuation)
    returnRow = trainingRow.translate(translator).lower().split(" ")
    vector = np.zeros(10000, dtype = np.int8)
    
    for word in returnRow:
        if word in dictionary:
            if BOWType == 'BagOfWords':
                vector[dictionary[word]] = np.int8(1)
            elif BOWType == 'Frequency':
                vector[dictionary[word]] += 1
    
    if BOWType == 'Frequency':
#         To accomodate for the fact that one of the rows has one word, d-gust-ing, and the lenght of the vector is zero.
#         Hence, the vector returns [Nan Nan Nan ... Nan Nan].  Now it returns [0 0 0 ... 0 0 0]
        vectorLength =  np.sum(vector)
        if vectorLength>0:
            vector = np.divide(vector, vectorLength)
            
    return vector

In [8]:
def createBagOfWordsMatrix(df, dictionary, BOWType):
    translator = str.maketrans('','', string.punctuation)
    
    if BOWType == 'BagOfWords':
        distancesArray = np.zeros((len(df), len(dictionary)), dtype = np.int8)
    elif BOWType == 'Frequency':
        distancesArray = np.zeros((len(df), len(dictionary)))
        
    for i in range(0, len(df)):
        vector = convertWordsToVector(df.iloc[i,0], dictionary, BOWType)
        distancesArray[i] = vector
#         print(np.sum(distancesArray[i]))
    return distancesArray
# trainingDF = 
# npArray = createBagOfWordsMatrix(readTxt('\yelp-train.txt'), createDictionary('yelp'), 'Frequency')
print(createBagOfWordsMatrix(readTxt('\yelp-train.txt'), createDictionary('yelp'), 'Frequency'))
# dfArray = pd.DataFrame(npArray)
# dfArray.to_csv(r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment3\Datasets\TESTING2.csv', header = False, sep=',', index = False)

[[ 0.05434783  0.02173913  0.00543478 ...,  0.          0.          0.        ]
 [ 0.03488372  0.03488372  0.0872093  ...,  0.          0.          0.        ]
 [ 0.05555556  0.03703704  0.04938272 ...,  0.          0.          0.        ]
 ..., 
 [ 0.03333333  0.075       0.04166667 ...,  0.          0.          0.        ]
 [ 0.08029197  0.03832117  0.05291971 ...,  0.          0.          0.        ]
 [ 0.02777778  0.05555556  0.         ...,  0.          0.          0.        ]]


In [15]:
def createReviewsTxt(df, dictionary):
    translator = str.maketrans('','',string.punctuation)
    
    
    for i in range(0, 2):
        reviewsString = ''
        trainingRow = df.iloc[i,0].translate(translator).lower().split(" ")
        for word in trainingRow:
            if word in dictionary:
                reviewsString += word + ' '
        print(reviewsString)
            
createReviewsTxt(readTxt('\yelp-train.txt'), createDictionary('yelp'))        
        

i cant believe i havent yelped about the place yet several months maybe over a year ago my husband read a newspaper article about the clover coffee maker and the one place in town that had managed to one i was skeptical as is my nature it cant be that much better right youre just saying its amazing because you want to talk about the new hot coffee shop you discovered right well maybe but i love this place and i dont think it has a whole lot to do with the clover they roast their own beans and they roast them way differently than that other ginormous coffee chain  all a light or medium roast never bitter never oily never yucky the coffee they make there is obviously the best but i send my husband in every week now to buy a pound of beans so that i can the same coffee at home add an edgy though sometimes intimidating seating area great local art which we bought off the wall and smiley sold cant wait to try out the downtown location 
best nights to go to postinos are mondays and tuesdays 

In [9]:
def testScores(predicted, actual):
    print('F1 Score:', f1_score(actual, predicted, average='macro'))
    print('Confusion Matrix:\n', confusion_matrix(actual, predicted))

In [11]:
def randomClassifier(dataset):
    print(dataset, 'Random Classifier')
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
    
    randomClassifier = DummyClassifier(strategy='uniform')
    randomClassifier.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = randomClassifier.predict(createBagOfWordsMatrix(trainingDF, dictionary))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = randomClassifier.predict(createBagOfWordsMatrix(validDF, dictionary))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = randomClassifier.predict(createBagOfWordsMatrix(testDF, dictionary))
    testScores(predictionsArray, testDF[1])
    
randomClassifier('yelp')

yelp Random Classifier
Training F1-Measure
F1 Score: 0.184123905128
Confusion Matrix:
 [[107 106  96 119  94]
 [124 129 115 137 136]
 [202 203 199 199 194]
 [503 478 484 497 506]
 [473 462 483 464 490]]
Validation F1-Measure
F1 Score: 0.176673463167
Confusion Matrix:
 [[14 15 23 18 14]
 [24 16 20 12 24]
 [25 35 37 34 33]
 [62 68 82 74 70]
 [58 48 70 71 53]]
Test F1-Measure
F1 Score: 0.18894982874
Confusion Matrix:
 [[ 27  29  25  33  29]
 [ 37  40  38  36  39]
 [ 63  54  60  65  58]
 [126 164 135 145 132]
 [122 127 133 136 147]]


In [12]:
def majorityClassifier(dataset):
    print(dataset, 'Majority Classifier')
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
    
    majorityClassifier = DummyClassifier(strategy='most_frequent')
    majorityClassifier.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = majorityClassifier.predict(createBagOfWordsMatrix(trainingDF, dictionary))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = majorityClassifier.predict(createBagOfWordsMatrix(validDF, dictionary))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = majorityClassifier.predict(createBagOfWordsMatrix(testDF, dictionary))
    testScores(predictionsArray, testDF[1])
    
# baselineClassifier('IMDB')
majorityClassifier('yelp')

yelp Majority Classifier
Training F1-Measure


C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


F1 Score: 0.104267004647
Confusion Matrix:
 [[   0    0    0  522    0]
 [   0    0    0  641    0]
 [   0    0    0  997    0]
 [   0    0    0 2468    0]
 [   0    0    0 2372    0]]
Validation F1-Measure
F1 Score: 0.105014749263
Confusion Matrix:
 [[  0   0   0  84   0]
 [  0   0   0  96   0]
 [  0   0   0 164   0]
 [  0   0   0 356   0]
 [  0   0   0 300   0]]
Test F1-Measure
F1 Score: 0.103923019985
Confusion Matrix:
 [[  0   0   0 143   0]
 [  0   0   0 190   0]
 [  0   0   0 300   0]
 [  0   0   0 702   0]
 [  0   0   0 665   0]]


In [15]:
def bernoulliNB(dataset):
    print(dataset)
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
        
    clf = BernoulliNB()
    clf.fit(createBagOfWordsMatrix(trainingDF, dictionary, 'Frequency'), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(trainingDF, dictionary, 'Frequency'))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(validDF, dictionary, 'Frequency'))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(testDF, dictionary, 'Frequency'))
    testScores(predictionsArray, testDF[1])
    
bernoulliNB('IMDB')

IMDB
Training F1-Measure
F1 Score: 0.871295845531
Confusion Matrix:
 [[6663  837]
 [1093 6407]]
Validation F1-Measure
F1 Score: 0.842861266874
Confusion Matrix:
 [[4293  707]
 [ 864 4136]]
Test F1-Measure
F1 Score: 0.835917809086
Confusion Matrix:
 [[10844  1656]
 [ 2442 10058]]


In [14]:
def decisionTree(dataset):
    print(dataset)
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
        
    clf = tree.DecisionTreeClassifier()
    clf.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(trainingDF, dictionary))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(validDF, dictionary))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(testDF, dictionary))
    testScores(predictionsArray, testDF[1])
    
decisionTree('yelp')

yelp
Training F1-Measure
F1 Score: 1.0
Confusion Matrix:
 [[ 522    0    0    0    0]
 [   0  641    0    0    0]
 [   0    0  997    0    0]
 [   0    0    0 2468    0]
 [   0    0    0    0 2372]]
Validation F1-Measure
F1 Score: 0.279822135371
Confusion Matrix:
 [[ 19  13  14  14  24]
 [ 13  11  17  24  31]
 [  8  21  40  58  37]
 [ 19  31  57 137 112]
 [ 15  17  31 110 127]]
Test F1-Measure
F1 Score: 0.276493554759
Confusion Matrix:
 [[ 24  24  27  31  37]
 [ 22  28  42  57  41]
 [ 11  33  59 133  64]
 [ 37  44  91 278 252]
 [ 25  35  60 237 308]]


In [15]:
def linearSVC(dataset):
    print(dataset)
    if dataset == 'yelp':
        dictionary = createDictionary('yelp')
        trainingDF = readTxt('\yelp-train.txt')
        validDF = readTxt('\yelp-valid.txt')
        testDF = readTxt('\yelp-test.txt')
    elif dataset == 'IMDB':
        dictionary = createDictionary('IMDB')
        trainingDF = readTxt('\IMDB-train.txt')
        validDF = readTxt('\IMDB-valid.txt')
        testDF = readTxt('\IMDB-test.txt')
        
    clf = LinearSVC()
    clf.fit(createBagOfWordsMatrix(trainingDF, dictionary), trainingDF[1])
    
    print('Training F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(trainingDF, dictionary))
    testScores(predictionsArray, trainingDF[1])
    
    print('Validation F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(validDF, dictionary))
    testScores(predictionsArray, validDF[1])
    
    print('Test F1-Measure')
    predictionsArray = clf.predict(createBagOfWordsMatrix(testDF, dictionary))
    testScores(predictionsArray, testDF[1])
    
linearSVC('yelp')

yelp
Training F1-Measure
F1 Score: 0.997870801179
Confusion Matrix:
 [[ 520    0    0    0    2]
 [   0  640    0    0    1]
 [   0    0  996    1    0]
 [   0    0    0 2454   14]
 [   0    0    0    2 2370]]
Validation F1-Measure
F1 Score: 0.413428426203
Confusion Matrix:
 [[ 36  23   8   9   8]
 [ 14  25  15  28  14]
 [  7  23  50  65  19]
 [ 10  11  32 170 133]
 [  4   9  18 102 167]]
Test F1-Measure
F1 Score: 0.401651918294
Confusion Matrix:
 [[ 56  33  15  19  20]
 [ 33  58  41  36  22]
 [ 12  42  77 118  51]
 [ 12  31  84 340 235]
 [ 15  12  40 237 361]]
